In [351]:
import pandas as pd
import numpy as np
from statsmodels.api import tsa
import statsmodels.api as sm
import plotly.express as px
from scipy.stats import boxcox
from statsmodels.tsa.tsatools import lagmat
from statsmodels.regression.linear_model import OLS
from scipy import stats

def acorr_breusch_godfrey(res, nlags=None, store=False):
    """
    Breusch-Godfrey Lagrange Multiplier tests for residual autocorrelation.

    Parameters
    ----------
    res : RegressionResults
        Estimation results for which the residuals are tested for serial
        correlation.
    nlags : int, default None
        Number of lags to include in the auxiliary regression. (nlags is
        highest lag).
    store : bool, default False
        If store is true, then an additional class instance that contains
        intermediate results is returned.

    Returns
    -------
    lm : float
        Lagrange multiplier test statistic.
    lmpval : float
        The p-value for Lagrange multiplier test.
    fval : float
        The value of the f statistic for F test, alternative version of the
        same test based on F test for the parameter restriction.
    fpval : float
        The pvalue for F test.
    res_store : ResultsStore
        A class instance that holds intermediate results. Only returned if
        store=True.

    Notes
    -----
    BG adds lags of residual to exog in the design matrix for the auxiliary
    regression with residuals as endog. See [1]_, section 12.7.1.

    References
    ----------
    .. [1] Greene, W. H. Econometric Analysis. New Jersey. Prentice Hall;
      5th edition. (2002).
    """

    x = np.asarray(res.resid).squeeze()
    if x.ndim != 1:
        raise ValueError("Model resid must be a 1d array. Cannot be used on"
                         " multivariate models.")
    exog_old = res.model.exog
    nobs = x.shape[0]
    if nlags is None:
        # TODO: Switch to min(10, nobs//5) after 0.12
        import warnings
        warnings.warn("The default value of nlags is changing.  After 0.12, "
                      "this value will become min(10, nobs//5). Directly set"
                      "nlags or period to silence this warning.",
                      FutureWarning)

        nlags = np.trunc(12. * np.power(nobs / 100., 1 / 4.))
        nlags = int(nlags)

    x = np.concatenate((np.zeros(nlags), x))

    xdall = lagmat(x[:, None], nlags, trim="both")
    nobs = xdall.shape[0]
    xdall = np.c_[np.ones((nobs, 1)), xdall]
    xshort = x[-nobs:]

    exog = xdall
    k_vars = exog.shape[1]

    resols = OLS(xshort, exog).fit()
    ft = resols.f_test(np.eye(nlags, k_vars, k_vars - nlags))
    fval = ft.fvalue
    fpval = ft.pvalue
    fval = float(np.squeeze(fval))
    fpval = float(np.squeeze(fpval))
    lm = nobs * resols.rsquared
    lmpval = stats.chi2.sf(lm, nlags)
    # Note: degrees of freedom for LM test is nvars minus constant = usedlags

    if store:
        res_store = ResultsStore()
        res_store.resols = resols
        res_store.usedlag = nlags
        return lm, lmpval, res_store
    else:
        return lm, lmpval

## Загрузка данных

Загрузите данные. В этот раз их не нужно чистить, вся первичная обработка уже произведена. Вся работа будет построена на одном временном ряде. Период данных $-$ неделя. Следующие несколько предложений $-$ описание происхождения данных для тех, кому любопытно, можно пропустить.

Ряд представляет из себя спотовые цены на аммиак на черноморском базисе. Базис $-$ это некоторая агрегированная точка, включающая в себя несколько различных портов. Это сделано для того, чтобы получить некоторую агрегированную цену региона, уменьшив количество входных данных. Для репрезентативности цены в базисе взвешиваются на объёмы торгов в каждом порту. Эта схема отражает то, как устроены мировые товарные рынки. На некоторые товары нет общей единой мировой цены (как, например, нефть марки Brent), но есть региональная цена. Для справки: cпотовые цены $-$ цены, по которым продаются реальные товары или реальные бумаги в данный момент времени на условиях немедленной доставки. Также цены товарных рынков обычно подчиняются принципам организации поставок. Например, данные цены использовали принцип FOB, который означает, что с момента пересечения поручней судна в порту отгрузки покупатель несёт на себе как риски, так и все расходы. Считается, что продавец в этот момент выполнил все свои обязательства.

#### Задание 1 (0.5 баллов)
Визуализируйте данные. Какие характерные особенности данных, которые помогут впоследствии построить модель, вы можете идентифицировать?

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Задание 2 (1 балл)

Как вы можете видеть, дисперсия временного ряда на различных участках неоднородна. Используйте какое-то преобразование (логарифмы, степени, показательные функции и прочие монотонные и обратимые преобразования преобразования), которое это немного подкорректирует. Безусловно, тут дисперсия кластеризуется, но этот эффект можно попытаться несколько уменьшить. Должно получиться что-то в духе картинки ниже, хотя визуально направления трендов могут различаться. (Например, если вы будете возводить в отрицательную степень). Абсолютно не обязательно, что сохранится исходный рисунок трендов.

Результат этого пункта необязательно использовать далее, можно работать с исходным рядом, подразумевая, что отсутствие преобразований будет заменено корректировками уже на отборе моделей (Так можно делать для ETS, например, когда вы хотите реализовать мультипликативную модель, что обсуждалось на лекции. Но лучше преобразовать. Для моделей ARIMA преобразование будет сделать необходимо, если возможно). Все прогнозы необходимо получить в исходных значениях (заказчику обычно нужен прогноз цены, а не экспоненты от степени цены), поэтому трансформацию нужно производить только для тренировочной части, а на прогнозах модели перед подсчётам метрики производить обратное преобразование. Обязательно учтите этот факт при написании заданий на кросс-валидацию.

In [216]:
# ༼ つ ◕_◕ ༽つ

#### Задание 3 (0.5 баллов)
Разделите выборку на тренировочную и валидационную. Валидационная $-$ последние 30 наблюдений, тренировочная $-$ всё остальное. 

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Задание 4 (3 балла)
Руководствуясь теми паттернами, которые вы выделили ранее, выберите 3-5 моделей-кандидатов из класса ETS. Кратко поясните, почему была выбрана каждая модель. На тренировочной части временного ряда обучите все модели-кандидаты. Отберите две лучших по любому информационному критерию из доступных.


In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Задание 5 (2 балла)
С помощью теста Бройша-Годфри протестируйте наличие автокорреляций в остатках каждой из моделей. Для этого воспользуйтесь функцией, определённой в начале тетрадки. Это слегка исправленная функция из statsmodels, модифицированная взаимодействовать с результатом оценки ETS-моделей, а именно с объектом, возвращаемым методом fit() (Как из sm.tsa.ExponentialSmoothing,  так и из sm.tsa.statespace.ExponentialSmoothing).  Визуализируйте автокорреляции остатков для каждой из моделей по аналогии с семинаром. Получилось ли добиться некоррелированных остатков хотя бы в одной модели? Если нет, то предположите почему.

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Задание 6 (1 балл)
Вне зависимости от результатов предыдущего пункта, оцените ошибку прогноза с помощью кросс-валидации любым из двух методов (сдвигающееся или расширяющееся окно). Аналогично оцените ошибку для наивной модели. Удалось ли вашей модели побить наивный прогноз?

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Задание 7 (2 балла)
Оставьте в тренировочной выборке только последние 400 наблюдений и повторите задания 4-6. Изменились ли результаты теста на наличие автокорреляций в остатках (если результат теста не изменился, то улучшилось ли значение p-value)? Улучшилось ли качество прогнозов модели? Если да, то как Вы думаете, почему?

In [ ]:
# ༼ つ ◕_◕ ༽つ